### Using sourmash pangenome and compare to Roary
Roary uses the output from Prokka, then it extracts the coding regions in nucleotide space, then translates these internally using bioperl

For sourmash, to compare:
Use the fna files (genomes), the ffn files (predicted transcripts, nucl), the faa files (predicted protein) that come from prokka. 
Use scale [1,10,50,100,1000] and k[21,31]
Do this for 2 species:
Lactobacillus amylovorus
Holdemanella porci

Adapt sourmash code for the thresholds Roary uses

srun --account=ctbrowngrp -p med2 -J roary -t 6:00:00 -c 10 --mem=50gb --pty bash

snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 10 --rerun-incomplete -k -s pangenome_refgenomes.smk -n